## 从全连接层到卷积层

- 两个原则
  - 平移不变性
  - 局部性
- 如果一个系统输入时不稳定的，输出时稳定的，就像吃饭和消化的问题，那么就可以 **用卷积来求这个系统的余量**
- f函数：不稳定的输入
- g函数：稳定的输出
- 周围像素点对当前像素点产生怎样的影响
- 一个像素点对周围像素点的试探
- **对全连接层使用平移不变性和局部性得到卷积层**
$$h_{i, j} = \sum_{a,b}v_{i,j,a,b}x_{i+a,j+b} \quad \Rightarrow \quad h_{i,j} = \sum_{a=-\Delta}^{\Delta}{\sum_{b=-\Delta}^{\Delta}{v_{a,b}x_{i+a,j+b}}}$$

## 交叉相关 VS 卷积
- 二维交差相关
$$y_{i, j} = \sum_{a=1}^{h}{\sum_{b=1}^{w}{w_{a,b}x_{i+a,j+b}}}$$
- 二维卷积
$$y_{i, j} = \sum_{a=1}^{h}{\sum_{b=1}^{w}{w_{-a,-b}x_{i+a,j+b}}}$$
- 由于对称性，在实际使用中没有区别
- 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
- 核矩阵大小时超参数

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

# 定义二维自相关
def corr2d(X:torch.Tensor, K:torch.Tensor) -> torch.Tensor:
    h, w = K.shape
    Y = torch.zeros(size=(X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

- 自定义卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

- 图像中的边缘检测

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([1.0, -1.0]).reshape(1, -1)
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

- 这个卷积核只能检测垂直边缘，不能检测水平边缘

In [8]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

- 可学习卷积核

In [ ]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)